# House Price Prediction (Synthetic Dataset)

This notebook includes: data loading, cleaning (handling missing values and messy types), encoding, scaling, model training (Random Forest), evaluation, and saving the trained model.

Files included in the project:
- `data/house_prices.csv` (synthetic, purposely unclean)
- `models/trained_model.pkl` (trained RandomForest model)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../data/house_prices.csv')
df.head()


## 1. Quick exploration


In [ ]:
print('Shape:', df.shape)
print('\nMissing per column:')
print(df.isnull().sum())
print('\nExample types:')
print(df.dtypes)


## 2. Data cleaning
Handle messy types and missing values. We'll:
- Convert 'unknown'/'N/A' to NaN
- Convert numeric columns to numeric
- Fill numeric NaNs with median
- Fill categorical NaNs with mode
- Strip extra spaces and normalize categories


In [ ]:
# Replace common bad entries with NaN
df.replace(['unknown','UNKNOWN','N/A','n/a','na'], np.nan, inplace=True)

# Convert numeric-like columns to numeric
num_cols = ['LotArea','YearBuilt','OverallQual','TotalBsmtSF','GrLivArea','FullBath','BedroomAbvGr','GarageCars','GarageArea','SalePrice']
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Clean categorical: strip and lower-case to normalize
cat_cols = ['Neighborhood','HouseStyle']
for c in cat_cols:
    df[c] = df[c].astype(str).str.strip().replace({'none':'', 'nan':np.nan})
    df[c] = df[c].replace('None', np.nan)
    df[c] = df[c].replace({'':np.nan})

# Now check missing again
print(df.isnull().sum())


## 3. Impute missing values
- Numeric => median
- Categorical => mode


In [ ]:
for c in num_cols:
    med = df[c].median()
    df[c].fillna(med, inplace=True)

for c in cat_cols:
    mode = df[c].mode()[0]
    df[c].fillna(mode, inplace=True)

print('After imputation:')
print(df.isnull().sum())


## 4. Encoding categorical variables
We'll use `LabelEncoder` for simplicity (you can switch to One-Hot Encoding later)


In [ ]:
le = LabelEncoder()
for c in cat_cols:
    df[c+'_enc'] = le.fit_transform(df[c])

features = ['LotArea','YearBuilt','OverallQual','TotalBsmtSF','GrLivArea','FullBath','BedroomAbvGr','GarageCars','GarageArea','Neighborhood_enc','HouseStyle_enc']
X = df[features]
y = df['SalePrice']
X.head()


## 5. Feature scaling
Scale numeric features with `StandardScaler` (categoricals already encoded)


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=features)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print('Train shape:', X_train.shape)


## 6. Train model
Using `RandomForestRegressor` for a robust baseline.


In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Save the model
with open('../models/trained_model.pkl','wb') as f:
    pickle.dump({'model':model,'scaler':scaler,'features':features}, f)

print('Model trained and saved to ../models/trained_model.pkl')


## 7. Evaluate


In [ ]:
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred, alpha=0.3)
plt.xlabel('Actual SalePrice')
plt.ylabel('Predicted SalePrice')
plt.title('Actual vs Predicted')
plt.show()


## 8. How to use the saved model
You can load `models/trained_model.pkl` and use it to predict on new data (remember to apply the same scaler and feature ordering).


In [ ]:
with open('../models/trained_model.pkl','rb') as f:
    saved = pickle.load(f)
saved.keys()
